In [2]:
from io import StringIO, BytesIO
from PIL import Image
from scipy.misc import imresize
import random

from sklearn.metrics import roc_auc_score
import numpy as np

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1" # only use GPU 1

In [3]:
import keras
from keras.models import Model
from keras import layers
from keras import backend as K
from keras.engine.topology import Layer
import tensorflow as tf

Using TensorFlow backend.


In [4]:
dataset_name = 'AmazonFashion6ImgPartitioned.npy'
dataset_dir = '../dataset/amazon/'
dataset = np.load(dataset_dir + dataset_name, encoding = 'bytes')

[user_train, user_validation, user_test, Item, usernum, itemnum] = dataset

## Data Process

In [5]:
def preprocess(img_s, mean=np.array([0.43, 0.47, 0.49]), std=np.array([1.0, 1.0, 1.0])):
    return (imresize(np.asarray(Image.open(BytesIO(img_s))), (224, 224, 3)) / 255 - np.array(mean)) / np.array(std)
class DataGenerator(keras.utils.Sequence):
    
    def __init__(self, source, batch_size=32, dim=(224, 224, 3)):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.source = source
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.source) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Sampling
        X = {'input_i': np.empty((self.batch_size, *self.dim)), 
             'input_j': np.empty((self.batch_size, *self.dim)),
             'input_user': np.empty((self.batch_size), dtype=int)}
        y = np.ones((self.batch_size), dtype=int)
        for k in range(self.batch_size):
            u = random.randint(0, len(self.source)-1)
            X['input_user'][k] = int(u)
            u_imgs = list(set([e[b'productid'] for e in self.source[u]]))
            
            i = u_imgs[random.randint(0, len(u_imgs)-1)]
            X['input_i'][k] = preprocess(Item[i][b'imgs'])
            j = random.randint(0, len(Item)-1)
            while j in u_imgs:
                j = random.randint(0, len(Item)-1)
            X['input_j'][k] = preprocess(Item[j][b'imgs'])
        return X, y


## Model

In [ ]:
w = 224
h = 224
input_shape = (w, h, 3)
dropout = 0.2
latent_d = 10 # latent dimension

user_num = 1000 # for test, this should be obtained from the dataset


def euclidean_distance(vects): 
    # L2 distance
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)




input_i = layers.Input(shape=input_shape, name="input_i") # image of item i
input_j = layers.Input(shape=input_shape, name="input_j") # image of item j
input_idx = layers.Input(shape=[1], name="input_user", dtype='int32') # idx of user u


# customer layer, learn the latent matrix of theta_u
class ThetaLayer(Layer):
    
    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='theta_u_matrix', 
                                      shape=(user_num, latent_d),
                                      initializer='uniform',
                                      trainable=True)
        super(ThetaLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        assert isinstance(x, list)
        item, u = x # u: user idx; item: visual feature of item 
        return K.dot(K.gather(self.kernel, u), item)

    def compute_output_shape(self, input_shape):
        print('input shape', input_shape)
        return (input_shape[0][0], 1)


# load the VGG pretrained on imagenet
def create_base_vgg(dropout):
    vgg = keras.applications.vgg19.VGG19(
        include_top=False, # whether to include the fc layers
        weights='imagenet', 
        input_tensor=None, 
        input_shape=input_shape, 
        pooling='avg',  # in my experience, gloable avg works better than flatten, need to check
        classes=1000)
    x = vgg.output
#     x = layers.Flatten(name='flatten')(x)
    x = layers.Dense(256, activation='relu', name='fc1')(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(latent_d, activation='relu', name='predictions')(x)
    x = layers.Reshape(target_shape=(latent_d, 1))(x)
    
    return Model(inputs = vgg.input, outputs = x, name="base_vgg")



# because we re-use the same instance `base_vgg`, theta_layer,
# the weights of the network
# will be shared across the two branches
base_vgg = create_base_vgg(dropout)
theta = ThetaLayer(name='theta_layer')

x1 = base_vgg(input_i)
x1 = theta([x1, input_idx])

x2 = base_vgg(input_j)
x2 = theta([x2, input_idx])

# distance = layers.Lambda(euclidean_distance,
#                   output_shape=eucl_dist_output_shape)([x1, x2])
            
distance = layers.Subtract(name='substract')([x1, x2])
distance = layers.Activation(activation='sigmoid', name='sigmoid')(distance)

model = Model([input_i, input_j, input_idx], distance)

# model.summary()

# -----------Evaluation---------------
# evaluation is different from training, 
# input of training: [user u, item i, item j]; 
# input of evaluation: [user u, item i]
# predict_score = layers.Activation(activation='sigmoid')(x1)
# evaluation_model = Model([input_i, input_idx], predict_score)
# evaluation_model.summary()

input shape [(None, 10, 1), (None, 1)]
input shape [(None, 10, 1), (None, 1)]


## Callbacks

In [20]:
class history(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [ ]:
batch_size = 128
epochs = 30
# loss function
def simple_BPR_loss(y_true, y_pred):
    return -K.sum(y_pred)

# first: freeze all convolutional layers, only train fc layers (which were randomly initialized)
# set trainable layers before model compile
for layer in base_vgg.layers[:-4]:
    layer.trainable = False
for layer in base_vgg.layers[-4:]:
    layer.trainable = True
# adam optimizer
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer=adam, loss=simple_BPR_loss)

training_generator = DataGenerator(user_train)
validation_generator = DataGenerator(user_validation)

model.fit_generator(generator=training_generator,
                   validation_data=validation_generator)
# model.fit(x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)
# evaluation_model.evaluate(x=None, y=None, batch_size=None, verbose=1, sample_weight=None, steps=None)


Epoch 1/1
